In [1]:
import sources.logins as lg
from time import sleep
import warnings
import pandas as pd
import os
import xlwings as xw
from datetime import datetime as dt
warnings.filterwarnings('ignore')
import pathlib
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
import math
from sqlite3 import connect

In [2]:

def login_exfm(driver_on = True):
    # generate key using fernet
    # key = Fernet.generate_key().decode()
    key = 'cLAiAdZU1U0sxWWK8sxhF0IWIBP4KrR3xhdu3x1EDQI='
    
    
    # login ExFM and download latest incompleted data
    name,pw = lg.save_id('exfm.txt',key)
    driver = lg.login_url('exfm',name,pw)
    
    
    while True:
        try:
            driver.find_element_by_xpath('//*[@id="RPR_SEARCH_LINK"]').click() # select RMA Search	
            break
        except Exception as e:
            print ('Wrong Username or Password. Try again...')
            name,pw = lg.save_id('exfm.txt',key)
            driver = lg.login_url('exfm',name,pw)
    
    d_type_menu = [
                    'Incompleted',
                    'History',
                    'Equipments',
                    'Customers',
                    'No Download'
                    
                    ]
    #border table
    print(f'\n{"_"*50}')
    print(f'{"|  No.|  Function": <49}|')
    print(f'|{"_"*48}|')
    for i in range(1,1+len(d_type_menu)):
        print(f'|{i: >3}  |  {d_type_menu[i-1]: <40}|')
    print(f'|{"_"*48}|') #bottom border
    while True:
        ind = str(input('Select Dataset to download: '))
        try:
            ind = int(ind)
        except:
            if ind.upper() == 'Q' or ind.upper() == 'QUIT':
                break
            else:
                print('Only accept number')
                continue
        if 0<ind<=len(d_type_menu):
            d_type = d_type_menu[ind-1]
            break
        else:
            print(f'Input number must be less than {len(d_type_menu)}\n')
    
    print(d_type)
    
    # driver.find_element_by_xpath('//*[@id="RPR_SEARCH_LINK"]').click() # select RMA Search
    
    if d_type == d_type_menu[0]: # Incompleted
        driver.find_element_by_xpath('//*[@id="sidEXPORT_CONSOLIDATED_BUTTON_IMAGE"]').click() #click download
    
    if d_type == d_type_menu[1]: # History
        Select(driver.find_element_by_xpath('//*[@id="sidIN_REPAIR_STATUS"]')).select_by_index(0)
        print('Select Status "All" and download.')
        driver.find_element_by_xpath('//*[@id="sidEXPORT_CONSOLIDATED_BUTTON_IMAGE"]').click() #click download
    
    if d_type == d_type_menu[2]: # Equipments
        driver.find_element_by_xpath('//*[@id="EQP_MGT_LINK"]').click() # Equipments
        sleep(0.5)
        driver.find_element_by_xpath('//*[@id="sidEXPORT_BUTTON_IMAGE"]').click()
    
    if d_type == d_type_menu[3]: # Customers
        driver.find_element_by_xpath('//*[@id="ACC_MGT_LINK"]').click() 
        sleep(0.5)
        driver.find_element_by_xpath('//*[@id="sidEXPORT_BUTTON_IMAGE"]').click()
    
    else:
        print('No Download')

    # check latest file
    i_wait = 0
    while True:
        try: # check file already
            file, ctime = lg.file_latest(folder_name='Downloads')
            if file.endswith('xls'):
                print (f'\nDonwload file {file} succesful at {ctime}')
                break
            else:
                print(f'Please wait for compressing file {file}...{i_wait}s')
                i_wait +=3
                sleep(3)
        except Exception as e: #waiting for download
            print(f'Please wait for dowloading...{i_wait}s')
            i_wait +=3
            sleep(3)
            if d_type == d_type_menu[-1]: break
    if driver_on:
        return driver
    else:
        #finish close driver
        driver.close()
    sleep(2)

def auto_accept(driver):
    WebDriverWait(driver, 5).until(EC.alert_is_present())
    driver.switch_to.alert.accept()
    
def export_all_translog_in_page(driver):
    for i in range(50):
        # translog button
        driver.find_element_by_xpath('//*[@id="sidTRANSLOG_BUTTON_IMAGE"]').click()
        p = driver.current_window_handle
        chwd = driver.window_handles
        for w in chwd:
            if(w!=p):
                driver.switch_to.window(w)
                break
        sn = driver.find_element_by_xpath('/html/body/div[1]/div[2]/table/tbody/tr[1]/td[1]').get_attribute('innerHTML')
        rma = driver.find_element_by_xpath('/html/body/div[1]/div[2]/table/tbody/tr[1]/td[2]').get_attribute('innerHTML')
        print(i+1,sn,rma)
        # click export
        driver.find_element_by_xpath('//*[@id="export"]').click()
        driver.close()
        sleep(0.5)
        driver.switch_to.window(p)

        # next button
        try:
            next_button = driver.find_element_by_xpath('//*[@id="sidSHOW_NEXT_REPAIR"]')
            next_button.click()
        except Exception as e:
            print(f"Finish all {i+1} jobs in this page.\n")
            break
            
def export_translog_by_sn(driver):
    
    # translog button
    driver.find_element_by_xpath('//*[@id="sidTRANSLOG_BUTTON_IMAGE"]').click()
    p = driver.current_window_handle
    chwd = driver.window_handles
    for w in chwd:
        if(w!=p):
            driver.switch_to.window(w)
            break
    sn = driver.find_element_by_xpath('/html/body/div[1]/div[2]/table/tbody/tr[1]/td[1]').get_attribute('innerHTML')
    rma = driver.find_element_by_xpath('/html/body/div[1]/div[2]/table/tbody/tr[1]/td[2]').get_attribute('innerHTML')
    
    # click export
    driver.find_element_by_xpath('//*[@id="export"]').click()
    driver.close()
    sleep(0.5)
    driver.switch_to.window(p)

    # next button
    try:
        next_button = driver.find_element_by_xpath('//*[@id="sidSHOW_NEXT_REPAIR"]')
        next_button.click()
    except Exception as e:
        print(f"Finish all {i+1} jobs in this page.\n")
        

In [3]:

driver = login_exfm()


Select account:
1   |   thongle
2   |   Nguyen
3   |   FFAP0104
0   |   CREATE NEW ACCOUNT
Select account by number: 1
thongle is selected to login
Removed Downloads


Create new folder: Downloads
Select Chrome Driver and press enter for authorize Certificate...

Typing your ID...
Typing your password...
Login sucessful!


__________________________________________________
|  No.|  Function                                |
|________________________________________________|
|  1  |  Incompleted                             |
|  2  |  History                                 |
|  3  |  Equipments                              |
|  4  |  Customers                               |
|  5  |  No Download                             |
|________________________________________________|
Select Dataset to download: 5
No Download
No Download
Please wait for dowloading...0s


In [4]:
#RMA search tab
driver.find_element_by_xpath('//*[@id="RPR_SEARCH_LINK"]').click()

In [5]:
#show all
driver.find_element_by_xpath('//*[@id="sidSEARCH_BUTTON_IMAGE"]').click()
sleep(0.5)
ttl = driver.find_element_by_xpath('//*[@id="TH_search_result"]/thead/tr[1]/th/div/span').get_attribute('innerHTML')
ttl = int(ttl.split('/')[1][:-1])
print(ttl)

366


In [6]:
# download all to Download Folder
pg = math.ceil(ttl/50)
for i in range(pg):
    first_row = i * 50
    #Fisrt row
    driver.find_element_by_xpath(f'//*[@id="search_result[{first_row}].sidRS_REPAIR_ID_data"]').click()
    # export translog
    print(f'Export for page {i + 1}/{pg}')
    export_all_translog_in_page(driver)
    # return
    driver.find_element_by_xpath('//*[@id="sidBACK_BUTTON_IMAGE"]').click()
    sleep(0.5)
    # next page
    driver.find_element_by_xpath('//*[@id="_eventcursornextpage_search_result"]').click()
    sleep(0.5)

Export for page 1/8
1 1C728K450 FMSV2023030040
2 4G361A481 FMSV2023030041
3 2V644K028 FMSV2023030060
4 1C741K218 FMSV2023030069
5 2V567K740 FMSV2023030079
6 2G348K408 FMSV2023040070
7 1G402K303 FMSV2023050025
8 4C591A001 FMSV2023050045L
9 1G391K564 FMSV2023050072
10 1G402K304 FMSV2023060044
11 1G403K110 FMSV2023060047
12 1C728K177 FMSV2023060048
13 2S101K411 FMSV2023060058
14 5G361K065 FMSV2023060068
15 2V567K687 FMSV2023070039
16 2G361K131 FMSV2023070056
17 1G391K315 FMSV2023070077
18 3V567K036 FMSV2023080002
19 1G374K042 FMSV2023080006
20 LG361K344 FMSV2023080008
21 7G391K123 FMSV2023080014
22 8G391K425 FMSV2023080015
23 6C692K127 FMSV2023080016
24 5C692K087 FMSV2023080018
25 1C607K008 FMSV2023080020
26 1S094A282 FMSV2023080022
27 3C692K101 FMSV2023080024
28 6G402K254 FMSV2023080036
29 1G391K524 FMSV2023080038
30 1G412K264 FMSV2023080050
31 2G402K022 FMSV2023080056
32 2G202K019 FMSV2023080057
33 1C309K265 FMSV2023080058
34 2G361K799 FMSV2023080061
35 1G361K957 FMSV2023080062
36 2G391

36 3C741K158 FMSV2023100005
37 8G391K040 FMSV2023100006
38 1C692K442 FMSV2023100007
39 LG361K013 FMSV2023100010
40 3C643K019 FMSV2023100011
41 5C692K035 FMSV2023100013
42 1G202K277 FMSV2023100014
43 1G403K172 FMSV2023100017
44 1C692K656 FMSV2023100018
45 2G391K630 FMSV2023100025
46 2G361K547 FMSV2023100027
47 7G391K045 FMSV2023100028
48 4G361K130 FMSV2023100029
49 RU039A048 FMSV2023100030
50 1B084K400 FMSV2023100031
Finish all 50 jobs in this page.

Export for page 7/8
1 2V567K691 FMSV2023100036
2 1C728K139 FMSV2023100038
3 6G403K047 FMSV2023100039
4 JG643K005 FMSV2023100041
5 LG361K352 FMSV2023100042
6 3D103L050 FMSV2023110004
7 1G402K296 FMSV2023110005
8 1C728K173 FMSV2023110006
9 1C692K065 FMSV2023110007
10 8C730K015 FMSV2023110008
11 1V567K251 FMSV2023110009
12 1C694K023 FMSV2023110010
13 8C730K003 FMSV2023110011
14 2C643K211 FMSV2023110012
15 7G390K013 FMSV2023110013
16 3G202K036 FMSV2023110015
17 20314060R FMSV2023110016
18 1G391K384 FMSV2023110017
19 9G402K763 FMSV2023110018
20 

In [ ]:
path = pathlib.Path().absolute()
today = dt.now().strftime('%y%m%d')
#rename Downloads to translog_today
os.rename(os.path.join(path,'Downloads'),os.path.join(path,f'translog_{today}'))

In [ ]:
os

In [ ]:
# download by sn
i = 0
for sn in sn_list['SERIAL_NO']:
    i+=1
    driver.find_element_by_xpath('//*[@id="sidIN_BODY_NO"]').clear()
    sleep(0.1)
    driver.find_element_by_xpath('//*[@id="sidIN_BODY_NO"]').send_keys(sn)
    sleep(0.5)
    #Fisrt row
    driver.find_element_by_xpath(f'//*[@id="search_result[0].sidRS_REPAIR_ID_data"]').click()
    # export translog
    sleep(0.5)
    export_translog_by_sn(driver)
    # return
    driver.find_element_by_xpath('//*[@id="sidBACK_BUTTON_IMAGE"]').click()
    sleep(0.5)
    print(f'{i}. Export translog for {sn}')

In [ ]:
sn_list

In [ ]:
# driver.find_element_by_xpath()

In [ ]:
# driver.find_element_by_xpath()

In [ ]:
# driver.find_element_by_xpath()

In [ ]:
# driver.find_element_by_xpath()

In [ ]:
# driver.find_element_by_xpath()

In [ ]:
# driver.find_element_by_xpath()

In [ ]:
# driver.find_element_by_xpath()

In [ ]:
# driver.find_element_by_xpath()

### Read excel file

In [ ]:
# before 22-Nov-23
standard_format = {'RMA':'','SN':'',
              'Receive':'','Receiver':'',
              'Inspection':'','Inspector':'',
              'Report':'','Reporter':'',
              'Verify':'','Verifier':'',
              'Quotation':'','Admin':'',
              'Confirmation':'','FSS':'',
              'Check_Stock':'','Stock_Keeper':'',
              'PO_Create':'','Admin2':'',
              'APAS repair':'','Admin3':'',
              'APAS approved':'','Admin4':'',
                
              }
def read_translog(df):
    job_translog = {'RMA':'','SN':'',
              'Receive':'','Receiver':'',
              'Inspection':'','Inspector':'',
              'Report':'','Reporter':'',
              'Verify':'','Verifier':'',
              'Quotation':'','Admin':'',
              'Confirmation':'','FSS':'',
              'Check_Stock':'','Stock_Keeper':'',
              'PO_Create':'','Admin2':'',
              'APAS repair':'','Admin3':'',
              'APAS approved':'','Admin4':'',
                
              }
    # sn = df['S/N'][0]

    job_translog['RMA'] = df['RMA#'][0]
    job_translog['SN'] = df['S/N'][0]
    for i in range(len(df)):
        # 1. Receive
        if df['Screen'][i]=='Shipping Incoming' and df['Type'][i]=='Completion':
            dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
            job_translog.update({'Receive':dtime.strftime('%Y-%m-%d'),'Receiver':df['User'][i]})

        # 2. Inspection
        if df['Screen'][i]=='Inspection' and df['Type'][i]=='Completion':
            dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
            job_translog.update({'Inspection':dtime.strftime('%Y-%m-%d'),'Inspector':df['User'][i]})

        # 3. Report Done
        if  df['Updated Item(After)'][i] in ('Status Info:"Waiting for Next Process Available"',''):
            dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
            job_translog.update({'Report':dtime.strftime('%Y-%m-%d'),'Reporter':df['User'][i]})

        # 4. Report Verified
        if  df['Updated Item(After)'][i] in ('Status Info:"Awating Parts billing"',''):
            dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
            job_translog.update({'Verify':dtime.strftime('%Y-%m-%d'),'Verifier':df['User'][i]})

        # 5. Quote Sent
        if df['Screen'][i]=='Parts Selection' and df['Type'][i]=='Completion':
            dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
            job_translog.update({'Quotation':dtime.strftime('%Y-%m-%d'),'Admin':df['User'][i]})

        # 6. Confirmation
        if df['Screen'][i]=='Authorization' and df['Type'][i]=='Completion':    
            dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
            job_translog.update({'Confirmation':dtime.strftime('%Y-%m-%d'),'FSS':df['User'][i]})

        # 7. Check stock
        if  'Waiting for Parts' in df['Updated Item(After)'][i] or 'Parts on Order' in df['Updated Item(After)'][i]:
            dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
            job_translog.update({'Check_Stock':dtime.strftime('%Y-%m-%d'),'Stock_Keeper':df['User'][i]})

        # 8. PO Created
        if  'Parts on Order' in df['Updated Item(After)'][i]:
            dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
            job_translog.update({'PO_Create':dtime.strftime('%Y-%m-%d'),'Admin2':df['User'][i]})

        # 9. APAS repair
        if  df['Screen'][i]=='Parts Selection' and 'Awaiting Internal Approval' in df['Updated Item(After)'][i]:
            dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
            job_translog.update({'APAS Repair':dtime.strftime('%Y-%m-%d'),'Verifier':df['Admin3'][i]})

        # 10. APAS Approved
    #     if 


    return job_translog



In [7]:
# new format
standard_format = {'RMA':'','SN':'',
              'Receive':'','Receiver':'',
              'Inspection':'','Inspector':'',
              'Quotation':'','Admin':'',
              'Confirmation':'','FSS':'',
              'Check_Stock':'','Stock_Keeper':'',
              'PO_Create':'','Admin2':'',
              'PO_Received':'','Admin3':'',
               
              }
def read_translog(df):
    job_translog = {'RMA':'','SN':'',
              'Receive':'','Receiver':'',
              'Inspection':'','Inspector':'',
              'Quotation':'','Admin':'',
              'Confirmation':'','FSS':'',
              'Check_Stock':'','Requester':'',
              'PO_Create':'','Admin2':'',
              'PO_Received':'','Admin3':'',
               
              }
    # sn = df['S/N'][0]

    job_translog['RMA'] = df['RMA#'][0]
    job_translog['SN'] = df['S/N'][0]
    for i in range(len(df)):
        # 1. Receive
        if df['Screen'][i]=='Shipping Incoming' and df['Type'][i]=='Completion':
            dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
            job_translog.update({'Receive':dtime.strftime('%Y-%m-%d'),'Receiver':df['User'][i]})

        # 2. Inspection
        if df['Screen'][i]=='Inspection' and df['Type'][i]=='Completion':
            dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
            job_translog.update({'Inspection':dtime.strftime('%Y-%m-%d'),'Inspector':df['User'][i]})

        # 3. Quote Sent
        if df['Screen'][i]=='Parts Selection' and df['Type'][i]=='Completion':
            dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
            job_translog.update({'Quotation':dtime.strftime('%Y-%m-%d'),'Admin':df['User'][i]})

        # 4. Confirmation
        if df['Screen'][i]=='Authorization' and df['Type'][i]=='Completion':    
            dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
            job_translog.update({'Confirmation':dtime.strftime('%Y-%m-%d'),'FSS':df['User'][i]})

        # 5. Check stock
        if  'Waiting for Parts' in df['Updated Item(After)'][i] or 'Parts on Order' in df['Updated Item(After)'][i]:
            dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
            job_translog.update({'Check_Stock':dtime.strftime('%Y-%m-%d'),'Requester':df['User'][i]})

        # 6. PO Created
        if  'Parts on Order' in df['Updated Item(After)'][i]:
            dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
            job_translog.update({'PO_Create':dtime.strftime('%Y-%m-%d'),'Admin2':df['User'][i]})

        # 7. PO Received
        if  'PO Received' in df['Updated Item(After)'][i]:
            dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
            job_translog.update({'PO_Received':dtime.strftime('%Y-%m-%d'),'Admin3':df['User'][i]})

        # 10. APAS Approved
    #     if 


    return job_translog

'231127'

In [13]:
# today = '231108'
today = dt.now().strftime('%y%m%d')
folder_name = f'translog_{today}'

files = os.listdir(folder_name)
rma_dict={}

full_translog =pd.DataFrame(columns=['RMA','SN',
                                      'Receive','Receiver',
                                      'Inspection','Inspector',
#                                       'Report','Reporter',
#                                       'Verify','Verifier',
                                      'Quotation','Admin',
                                      'Confirmation','FSS',
                                      'Check_Stock','Requester',
                                      'PO_Create','Admin2',
                                      'PO_Received','Admin3'
#                                       'APAS approved','Admin4',
                                      
                                        ])

for file in files:
#     file = 'Downloads/export (99).csv'
    df = pd.read_csv(f'{folder_name}/{file}',encoding = 'utf-8')
    try:
#         display(standard_format)
        trans_log = read_translog(df)
        rma = trans_log['RMA']
        rma_dict.update({rma:file})
    except Exception as e:
        print(e)
        
            
    full_translog = full_translog.append({
        'RMA':trans_log['RMA'],'SN':trans_log['SN'],
          'Receive':trans_log['Receive'],'Receiver':trans_log['Receiver'],
          'Inspection':trans_log['Inspection'],'Inspector':trans_log['Inspector'],
#           'Report':trans_log['Report'],'Reporter':trans_log['Reporter'],
#           'Verify':trans_log['Verify'],'Verifier':trans_log['Verifier'],
          'Quotation':trans_log['Quotation'],'Admin':trans_log['Admin'],
          'Confirmation':trans_log['Confirmation'],'FSS':trans_log['FSS'],
          'Check_Stock':trans_log['Check_Stock'],'Requester':trans_log['Requester'],
          'PO_Create':trans_log['PO_Create'],'Admin2':trans_log['Admin2'],
          'PO_Received':trans_log['PO_Received'],'Admin3':trans_log['Admin3']
#           'APAS approved':trans_log['APAS approved'],'Admin4':trans_log['APAS approved']
    },ignore_index=True)
    trans_log=[]
print('Done')

Done


In [ ]:
full_translog

In [15]:
names = []
# headers = ['Receiver','Inspector','Reporter','Verifier',
#            'Admin','FSS','Stock_Keeper','Admin2','Admin3','Admin4']
headers = ['Receiver','Inspector','Admin','FSS','Requester','Admin2','Admin3']
for header in headers:
    for name in list (full_translog[header].unique()):
        if name not in names:
            names.append(name)
names.remove('')
print(len(names),names)

11 ['thongle', 'thang', 'Nguyen', 'nguyenminh', 'tranminh', 'FFAP0070', 'hoang', 'FFAP0104', 'FFAP0112', 'FFAP0062', 'FFAP0066']


In [16]:
path = pathlib.Path().absolute()
today = dt.now().strftime('%y%m%d')
# today
file_name = os.path.join(path,'inspection',f'translog_{today}.xlsx')
full_translog.to_excel(file_name,index=False)

In [17]:

wb = xw.Book(file_name)
sh = wb.sheets('Sheet1')
sh.autofit()

In [18]:
sh_new = wb.sheets.add()
sh_new.name = 'Summary'
sm = wb.sheets('Summary')

In [38]:
#print id
i = 0
for name in names:
    sm.range('A3').offset(i,0).value = name
    i+=1
#print headers
i = 0
for header in headers:
    sm.range('C1').offset(0,i).value = i
    sm.range('C2').offset(0,i).value = header
    i+=1
sm.range('C3').value = '=COUNTIF(OFFSET(Sheet1!D:D,0,C$1),$A3)'
sm.range('C3:I13').formula = sm.range('C3').formula

In [36]:
sm.clear_contents()